# Does The Sampling Method Matter?
#### Author: JP Melo

In this file we explore how different sampling method change the convergence speed and results.

### Imports

In [1]:
from derpinns.nn import *
from derpinns.utils import *
from derpinns.trainer import *
import torch

/Users/josemelo/Desktop/master/tesis/codes/.conda/lib/python3.11/site-packages/kfac/base_preconditioner.py:15: UserWarning: NVIDIA Apex is not installed or was not installed with --cpp_ext. Falling back to PyTorch flatten and unflatten.
  from kfac.distributed import get_rank


## Parameters

In [2]:
# Fix seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Global parameters
assets = 2

nn_shape = "64x3"               
device = torch.device("cpu") 
dtype = torch.float32

# Create dataset to traing over
batch_size = 500
total_iter = 1_000
boundary_samples = 20_000
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

# Define option valuation params
params = OptionParameters(
    n_assets=assets,
    tau=1.0,
    sigma=np.array([0.2] * assets),
    rho=np.eye(assets) + 0.25 * (np.ones((assets, assets)) - np.eye(assets)),
    r=0.05,
    strike=100,
    payoff=payoff
)

## Training

### With PRN

In [3]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "pseudo"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

Adam training: 100%|██████████| 1000/1000 [02:53<00:00,  5.78it/s, Interior=0.000741, Boundary=0.000578, Initial=0.003869, Total=0.005189, Max Error=111.0753784180, L2 Error=0.1726528555]


In [4]:
with_pseudo = trainer.closure.get_state()
plot_loss(with_pseudo, smooth=True, smooth_window=10)

with_pseudo_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_pseudo_results*100)

L2 Error:  9.545748


## With Sobol

In [5]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "Sobol"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

Adam training: 100%|██████████| 1000/1000 [02:50<00:00,  5.88it/s, Interior=0.000759, Boundary=0.000654, Initial=0.005610, Total=0.007022, Max Error=85.5677490234, L2 Error=0.1341786534]


In [6]:
with_sobol = trainer.closure.get_state()
plot_loss(with_sobol, smooth=True, smooth_window=10)

with_sobol_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_sobol_results*100)

L2 Error:  3.9504097


## With Halton

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "Halton"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

In [ ]:
with_halton = trainer.closure.get_state()
plot_loss(with_halton, smooth=True, smooth_window=10)

with_halton_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_halton_results*100)

## Residual Based Adaptive Sampling

In [21]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "Sobol"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = ResidualBasedAdaptiveSamplingDimlessBS(sampler=sampler, k=0.5, c=2, seed=0)\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

Adam training: 100%|██████████| 1000/1000 [03:06<00:00,  5.37it/s, Interior=0.001247, Boundary=0.000901, Initial=0.004302, Total=0.006450, Max Error=25.0466308594, L2 Error=0.0426526554]


In [23]:
with_ra = trainer.closure.get_state()
plot_loss(with_ra, smooth=True, smooth_window=10)

with_ra_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_ra_results*100)

L2 Error:  16.175331


9.5457487/3.9504097/11.257896

### Compare runs

In [ ]:
compare_loss_histories(
    [with_pseudo,with_sobol,with_halton, with_ra],
    ["Pseudo", "Sobol","Halton","RAS"],
    smooth=True,
    smooth_window=50,
)

The choice of sampling method has an impact in the final achieved loss. Contrary to the intuition, the pseudo random numbers outperform all methods.

In [20]:
print("Boundary: ", (1 - with_sobol['boundary_loss'][-1]/with_pseudo['boundary_loss'][-1])*100)
print("Interior: ", (1 - with_sobol['interior_loss'][-1]/with_pseudo['interior_loss'][-1])*100)
print("Initial: ", (1 - with_sobol['initial_loss'][-1]/with_pseudo['initial_loss'][-1])*100)


print("Boundary: ", (1 - with_ra['boundary_loss'][-1]/with_pseudo['boundary_loss'][-1])*100)
print("Interior: ", (1 - with_ra['interior_loss'][-1]/with_pseudo['interior_loss'][-1])*100)
print("Initial: ", (1 - with_ra['initial_loss'][-1]/with_pseudo['initial_loss'][-1])*100)

Boundary:  -13.00976363314117
Interior:  -2.3839385816156966
Initial:  -44.97353796934649
Boundary:  29.284271976729357
Interior:  -29.0981755164903
Initial:  25.16444824948164
